# Step 4: Post Process

In this part, we will show how to post process the results of qfold

In [ ]:
import math
import numpy as np
import random

from copy import Error


In [ ]:
protein_name = 'glycylglycine'
aminoacids = 'GG'
number_bits_to_discretize_angles_range = 4
protein_id = 0

input_filename = 'inputRotations'
output_filename = 'outputRotations'
#precalculated_energies_path = './precalculated_energies/'

basis = '6-31g'
energy_method = 'mp2'
method_rotations_generation = 'minifold'
execution_mode = 'simulation'
#beta_type = 'fixed'
#beta = 10

#provider = IBMQ.get_provider(hub='ibm-q-research', group='Roberto-Campos-Or', project='main')
#backend = provider.get_backend('ibmq_casablanca')

#initial_step = 2
#final_step = 10

#number_repetitions_real_mode = 1
#w_real_mode = 20
#ibmq_shots = 8192
#number_repetitions_ibmq = 1
#number_repetitions_ibmq_zero_beta = 2
#betas = [ 0.1, 1]

precision_solution = 0.9

#n_angles = (len(aminoacids) -1)*2
#move_id_len = int(np.ceil(np.log2(n_angles)))
#n_qubits_ancilla = 7

#kappa = 1

#annealing_schedule = 'geometric'
#alpha = 0.9

default_value_tts = 99999

In [ ]:
def calculate_tts_from_probability_matrix(probabilities_matrix, index_min_energy, step, precision_solution):

    p_t = 0
    # if the index of min energy calculated by psi 4 is in the results of metropolis, p_t is extracted
    # else, the p_t is set to a very small value close to 0 (not 0 to avoid inf values)
    if index_min_energy in probabilities_matrix.keys():
        p_t = probabilities_matrix[index_min_energy]
    else:
        p_t = 0

    result = 0
    # Result is the calculated TTS
    if p_t >= 1:
        result = 1
    elif p_t == 0:
        result = default_value_tts
    else:
        result = step * (math.log10(1-precision_solution)/(math.log10(1-p_t)))

    return result

In [ ]:
def get_selected_position_and_confidence(real_counts):

    max_value = 0
    position = 0
    confidence = 0

    for key in real_counts.keys():
        if real_counts[key] > max_value:
            max_value = real_counts[key]
            position = key
            confidence = real_counts[key]

    return [position, confidence]

In [ ]:
def create_input_file(protein, protein_id):

        inputFile = open(input_filename+'.dat', 'w')

        inputFile.write('molecule ' + protein + '{\n')

        if protein_id == -1:
            inputFile.write(' pubchem: '+ protein+'\n')
        else:
            inputFile.write(' pubchem: '+ protein_id+'\n')

        inputFile.write('}\n\n')

        inputFile.write('set basis ' +  basis + '\n')
        inputFile.write('set reference rhf\n')
        inputFile.write("energy('" + energy_method + "')\n")

        inputFile.close()

In [ ]:
def parse_psi_output_file(protein):

    atomId = 0
    protein_id = -1
    with open(output_filename+'.dat', 'r') as filehandle:

        isDataLine = False
        isInfoLine = False
        atoms = []
        for line in filehandle:

            #if line is an empty string after reading data
            if isDataLine and line.isspace():
                break
            
            # Data has ------ and it is necessary to avoid it
            if isDataLine and not '--' in line:

                lineChunks = line.split()
                atoms += [atom.Atom(atomId, lineChunks[0], float(lineChunks[1]), float(lineChunks[2]), float(lineChunks[3]), float(lineChunks[4]))]
                atomId += 1

            if isInfoLine and not 'Chemical ID' in line:
                protein_id = line.split()[0]
                break

            if 'Center' in line:
                isDataLine = True

            if 'Chemical ID' in line:
                isInfoLine = True
                    
    return [atoms, protein_id]

In [ ]:
def getAtomsFromProtein(protein, protein_id):

    #create input file
    create_input_file(protein, protein_id)

    #execute psi4
    execute_psi_command()

    #read/parse outputfile
    [atoms, protein_id] = parse_psi_output_file(protein)

    # if protein_id is not -1 means that psi4 was not able to find the protein but multiples ids for the protein
    # the solution is to create an input file with the name and the id
    if protein_id != -1:
        create_input_file(protein, protein_id)
        execute_psi_command()
        [atoms, protein_id] = parse_psi_output_file(protein)

    if atoms == []:
        raise Error('No atoms have been found!')

    return atoms

In [ ]:
def distance(atom, atom2):
        return np.sqrt((atom.x-atom2.x)**2+(atom.y-atom2.y)**2+(atom.z-atom2.z)**2)

In [ ]:
def is_proline_N(atom):

    carbon_ring = []

    if atom.element != 'N' or len(atom.linked_to_dict['C']) != 2 or len(atom.linked_to_dict['H'])!=1:
        return False
    else:
        carbons = atom.linked_to_dict['C']
        if len(carbons[0].linked_to_dict['C']) == 1  and len(carbons[0].linked_to_dict['N']) == 1 and len(carbons[1].linked_to_dict['C']) == 2 and len(carbons[1].linked_to_dict['N']) == 1:
            current_carbon = carbons[0]
            ending_carbon = carbons[1]
        elif len(carbons[1].linked_to_dict['C']) == 1  and len(carbons[1].linked_to_dict['N']) == 1 and len(carbons[0].linked_to_dict['C']) == 2 and len(carbons[0].linked_to_dict['N']) == 1:
            current_carbon = carbons[1]
            ending_carbon = carbons[0]
        else:
            return False
        
        for _ in range(2):
            carbon_ring.append(current_carbon)
            current_carbon = (current_carbon.linked_to_dict['C'][0] if current_carbon.linked_to_dict['C'][0] not in carbon_ring else current_carbon.linked_to_dict['C'][1])
            if len(current_carbon.linked_to_dict['C']) != 2 or len(current_carbon.linked_to_dict['N']) != 0 or len(current_carbon.linked_to_dict['O']) != 0 or len(current_carbon.linked_to_dict['H']) != 2:
                return False
            
        return (True if current_carbon in ending_carbon.linked_to else False)

In [ ]:
def main_chain_builder(self, nitrogen_starts, aminoacids):
        '''Takes all the nitrogens that are only connected to a single C and returns the backbone of the protein'''
        best_chains = []
        len_best_chain = 0
        for nitro in nitrogen_starts:
            candidate_chain = []
            nit = nitro

            for amino_index in range(len(aminoacids)):

                aminolist = []
                aminolist.append(nit)

                # Searching for C-alpha
                carbons = nit.linked_to_dict['C']
                carbons_not_in_chain = [carbon for carbon in carbons if (carbon not in candidate_chain and carbon not in aminolist)]
                if (len(carbons_not_in_chain)==1 and aminoacids[amino_index] != 'P'):
                    car_alpha = carbons_not_in_chain[0]
                    aminolist.append(car_alpha)
                elif (len(carbons_not_in_chain)==2 and aminoacids[amino_index] == 'P'):
                    car_alpha = (carbons_not_in_chain[0] if (len(carbons_not_in_chain[0].linked_to_dict['N']) == 1 and len(carbons_not_in_chain[0].linked_to_dict['C']) == 2 and len(carbons_not_in_chain[0].linked_to_dict['H']) == 1) else carbons_not_in_chain[1])
                    aminolist.append(car_alpha)
                else:
                    break

                # Searching for Carboxy
                carbons = car_alpha.linked_to_dict['C']
                carboxys_not_in_chain = [carbon for carbon in carbons if (carbon not in candidate_chain and carbon not in aminolist and len(carbon.linked_to_dict['O']) > 0)]
                if amino_index+1 < len(aminoacids):
                    carboxys_not_in_chain = [carbox for carbox in carboxys_not_in_chain if len(carbox.linked_to_dict['N']) > 0]
                if len(carboxys_not_in_chain)==1:
                    carbox = carboxys_not_in_chain[0]
                    aminolist.append(carbox)
                else:
                    break

                #We have a full aminoacid, so we save it to the candidate list
                candidate_chain += aminolist

                # Searching for next aminoacid Nitrogen
                nitrogens = carbox.linked_to_dict['N']
                nitrogens_not_in_chain = [n for n in nitrogens if (n not in candidate_chain and n not in aminolist)]
                if len(nitrogens_not_in_chain)==1:
                    nit = nitrogens_not_in_chain[0]
                else:
                    break

            # Is the found chain longer than the one we already had?
            if len(candidate_chain) > len_best_chain:
                len_best_chain = len(candidate_chain)
                best_chains = [candidate_chain]
            elif len(candidate_chain) == len_best_chain:
                best_chains.append(candidate_chain)
            else: 
                pass

        if len(best_chains) != 1 or len(best_chains[0])//3 != len(aminoacids):
            raise ValueError('There should be a single lengthy chain!', best_chains, nitrogen_starts)
        else:
            return best_chains[0]

In [ ]:
def calculate_atom_connection(atoms, aminoacids):

    #Let us first map the topology. Currently cost is O(N^2). Some other algorithm could be desirable
    for at1 in atoms:
        for at2 in atoms:
            if at1 != at2:
                if at1.element != 'H' and at2.element != 'H' and distance(at1,at2)<2  and (at1 not in at2.linked_to): 
                    at1.linked_to = [at2] + at1.linked_to 
                    at2.linked_to = [at1] + at2.linked_to

                elif at1.element != 'H' and at2.element == 'H' and distance(at1,at2)<1.3  and (at1 not in at2.linked_to):
                    at1.linked_to = [at2] + at1.linked_to 
                    at2.linked_to = [at1] + at2.linked_to

    # Next we give an structure to each linked_to list
    for at in atoms:
        at.linked_to_dict = {'N': [], 'O': [], 'C': [], 'H': [], 'Other': []}
        for at1 in at.linked_to:
            if at1.element == 'N':
                at.linked_to_dict['N'].append(at1)
            elif at1.element == 'O':
                at.linked_to_dict['O'].append(at1)
            elif at1.element == 'C':
                at.linked_to_dict['C'].append(at1)
            elif at1.element == 'H':
                at.linked_to_dict['H'].append(at1)
            else:
                at.linked_to_dict['Other'].append(at1)

    #plotting(list_of_atoms = atoms, title = 'Peptide_plot')

    #make a list of nitrogen atoms where one could start the main chain
    nitrogen_starts = []

    # For any aminoacid except proline
    if aminoacids[0] != 'P':
        for at in atoms:
            # This allows to identify any initial N to start except for Proline which has a weird structure
            if at.element == 'N' and len(at.linked_to_dict['C']) == 1 and len(at.linked_to_dict['H'])==2:
                nitrogen_starts.append(at)

    # For the protein starting at proline
    elif aminoacids[0] == 'P':
        for at in atoms:
            # This allows to identify any initial N to start except for Proline which has a weird structure
            if at.element == 'N' and is_proline_N(at):
                nitrogen_starts.append(at)

    # Find main_chain
    backbone = main_chain_builder(nitrogen_starts, aminoacids)

    # Name the atoms
    for (atom,i) in zip(backbone, range(len(backbone))):
        if atom.element == 'N' and (i % 3 == 0):
            atom.c_type = 'N_backbone'
        elif atom.element == 'C' and (i % 3 == 1) and (atom.linked_to_dict['O'] == []):
            atom.c_type = 'C_alpha'
        elif atom.element == 'C' and (i % 3 == 2) and (atom.linked_to_dict['O'] != []):
            atom.c_type = 'Carboxy'
        else:
            raise TypeError('The atom', atom.element, 'does not fulfill the requirements to be part of the backbone')

    return atoms, backbone

In [ ]:
def calculate_structure(self, atoms, aminoacids, init_method, bits, backbone, phi_positions, psi_positions):

    phis_initial_rotation = []
    psis_initial_rotation = []
    rotation_steps = pow(2, int(bits))

    psi_angles_psi4 = [self.tools.calculateAngle(backbone[3*j:3*j+4],'psi') for j in range(len(backbone)//3 - 1)]
    phi_angles_psi4 = [self.tools.calculateAngle(backbone[3*j-1:3*j+3],'phi') for j in range(1, len(backbone)//3)]

    
    #Set angles to 0. PSI4 returns the optimal angles for the protein, so it is necessary to set these angles to 0
    atoms = self.flat_protein(atoms, backbone, phi_angles_psi4, psi_angles_psi4)
    
    #random between -π and π
    if init_method == 'random':
        for _ in range(len(phi_angles_psi4)):

            phis_initial_rotation.append(random.uniform(-math.pi, math.pi))
            psis_initial_rotation.append(random.uniform(-math.pi, math.pi))

    #minifold
    elif init_method == 'minifold':

        mfold = minifold.Minifold(self.model_path, self.window_size, self.max_aa_length)
        angles = mfold.predictAngles(aminoacids)

        for angle in angles:

            phis_initial_rotation.append(angle[0])
            psis_initial_rotation.append(angle[1])


    # rotate to the initial position
    for index in range(len(phis_initial_rotation)):

        self.tools.rotate(angle_type = 'psi', angle = psis_initial_rotation[index], starting_atom = backbone[3*index+2], backbone = backbone)
        self.tools.rotate(angle_type = 'phi', angle = phis_initial_rotation[index], starting_atom = backbone[3*index+4], backbone = backbone)

    # rotate to the selected position
    for index in range(len(phi_positions)):

        self.tools.rotate(angle_type = 'psi', angle = (psis_initial_rotation[index]/rotation_steps) * 2*math.pi, starting_atom = backbone[3*index+2], backbone = backbone)
        self.tools.rotate(angle_type = 'phi', angle = (phis_initial_rotation[index]/rotation_steps) * 2*math.pi, starting_atom = backbone[3*index+4], backbone = backbone)

    return atoms

In [ ]:
def get_energy_configuration_from_position(position):

    energy = 0

    # calculate the structure (energy and configuration) of the protein from the position calculated by metropolis algorithms
    # it is possible to know the protein structure because it has the initial position and how many degrees was rotated (position * number of rotation bits)
    # First we calculate all the angles. Psi uses the first atom from the next aminoacid, whereas phi uses the last from the previous        

    # first half of position string is phi positions and the other half is psi positions
    phi_positions = position[:int(len(position)/2)]
    psi_positions = position[int(len(position)/2):]

    # get atoms
    atoms = getAtomsFromProtein(protein_name, protein_id)
    atoms, backbone = calculate_atom_connection(atoms, aminoacids)

    atoms = calculate_structure(atoms, aminoacids, method_rotations_generation, number_bits_to_discretize_angles_range, backbone, phi_positions, psi_positions)
    energy = calculate_energy_of_rotation(atoms)

    configuration = convert_atoms_to_configuration(atoms)

    return [energy, configuration]

In [ ]:
dict_probabilities_matrix = {} #read from json
index_min_energy = 0 #read from json
q_accumulated_tts = []
min_q_tts = {'step': 0, 'value': -1}
min_c_tts = {'step': 0, 'value': -1}

if execution_mode == 'simulation':

    for step, probabilities_matrix in dict_probabilities_matrix.items():

        ###### Accumulated values Quantum Metropolis ######
        q_tts = calculate_tts_from_probability_matrix(probabilities_matrix, index_min_energy, step, precision_solution)

        q_accumulated_tts.append(q_tts)     
        if q_tts < min_q_tts['value'] or min_q_tts['value'] == -1: min_q_tts.update(dict(value=q_tts, step=step))


# in real mode it is not necessary to execute the loop (there is only one step/w) so it breaks the loop
if execution_mode == 'real':

    quantum_success = False
    classical_success = False

    [quantum_selected_position, quantum_confidence] = get_selected_position_and_confidence(real_q_counts)
    [quantum_energy, quantum_configuration] = get_energy_configuration_from_position(quantum_selected_position)

    if quantum_selected_position == index_min_energy: quantum_success = True 
    quantum_stats = {'confidence':quantum_confidence, 'success':quantum_success, 'energy':quantum_energy, 'configuration':quantum_configuration}
    
    # if the classical position is equal than the quantum, it is not necessary to recalculate the configuration and energy, it is the same
    if classical_selected_position == quantum_selected_position:
        classical_energy = quantum_energy
        classical_configuration = quantum_configuration
    else:
        [classical_energy, classical_configuration] = get_energy_configuration_from_position(classical_selected_position, tools.args)
    
    if classical_selected_position == index_min_energy: classical_success = True 
    classical_stats = {'confidence': classical_confidence, 'success': classical_success, 'energy':classical_energy, 'configuration':classical_configuration}

    write_real_results(initialization_stats, quantum_stats, classical_stats)

    min_q_tts['value'] = quantum_confidence
    min_q_tts['success'] = quantum_success
    min_c_tts['value'] = classical_confidence
    min_c_tts['success'] = classical_success

    break

In [ ]:
print("<i> CLASSICAL METROPOLIS: Time for", step, "steps: %s seconds" % (time.time() - start_time))
c_tts = calculate_tts_from_probability_matrix(probabilities_matrix, index_min_energy, step, precision_solution)

#### create json files ####
if execution_mode == 'simulation':

    ###### Accumulated values Classical Metropolis ######
    c_accumulated_tts.append(c_tts)
    if c_tts < min_c_tts['value'] or min_c_tts['value'] == -1: min_c_tts.update(dict(value=c_tts, step=step))
    
    if step == final_step - 1: 
        # plot data
        #tools.plot_tts(q_accumulated_tts, c_accumulated_tts, tools.config_variables['initial_step'])

        # generate json data
        final_stats = {'q': min_q_tts, 'c': min_c_tts}
        write_tts(q_accumulated_tts, c_accumulated_tts, initialization_stats, final_stats)

elif execution_mode == 'experiment':

    p_t = experiment_result_matrix['betas=betas']['raw']['00']/ibmq_shots
    q_tts = calculateTTS('precision_solution', step, p_t)
    
    ###### Accumulated values Quantum Metropolis ######
    q_accumulated_tts.append(q_tts)     
    if q_tts < min_q_tts['value'] or min_q_tts['value'] == -1: min_q_tts.update(dict(value=q_tts, step=step))

    ###### Accumulated values Classical Metropolis ######
    c_accumulated_tts.append(c_tts)
    if c_tts < min_c_tts['value'] or min_c_tts['value'] == -1: min_c_tts.update(dict(value=c_tts, step=step))

    write_experiment_results(initialization_stats, experiment_result_matrix, execution_stats, measures_dict)